In [107]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import scipy as sp
import tmm_pandas as tmm_pd
import tmm_core as tmm
from matplotlib import pyplot as plt

In [2]:
class Material:
    def __init__(self, path, name, wavelengths):
        f = pd.read_csv(path)
        wv_raw = np.array(f.wv)
        nc = np.array(f.n+f.k*1j)  # complex refractive index
        self.name = name
        self.f = interp1d(wv_raw, nc, kind='cubic')
        self.min_wv = min(wv_raw)
        self.max_wv = max(wv_raw)
        self.df = pd.Series(data=nc, index=wv_raw, name=name)
        # self.df = self.df.reindex(wavelengths)
        # self.df = self.df.interpolate('spline', order=3)
        self.interp(wavelengths)

    def interp(self, wavelengths):
        wavelengths = wavelengths[np.nonzero(np.logical_and(wavelengths > self.min_wv, wavelengths < self.max_wv))]
        nc = self.f(wavelengths)
        self.df = pd.DataFrame(nc,wavelengths, [self.name])

In [3]:
class Model:
    def __init__(self):
        self.wavelength = np.arange(300, 1700, 1)
        self.increment = .2
        self.index_array = np.array([])
        self.materials = []
        self.mat_df = pd.DataFrame([], self.wavelength)
        self.data = {}
        self.thicknesses = [sp.inf, 0, sp.inf]
        self.layers = []

        self.add_material("TCO", './Materials/Semiconductor/TCO.csv')
        self.add_material("CdSe", './Materials/Semiconductor/CdSe.csv')
        self.add_material("CdTe", './Materials/Semiconductor/CdTe.csv')
        self.add_material("Air", './Materials/Semiconductor/Air.csv')
        self.add_material("BK7", './Materials/Dielectric/BK7.csv')
        self.add_material("SLG", './Materials/Dielectric/SLG.csv')
        self.add_material("SS TCO", './Materials/Dielectric/Sunnyside TCO.csv')
        self.add_material("SiO2", './Materials/Dielectric/SiO2-Jaw.csv')
        
    def add_material(self, film, path):
        if film not in self.mat_df:
            mat = Material(path, film, self.wavelength)
            self.materials.append(mat)
            self.mat_df = self.mat_df.join(mat.df)

    def set_wavelength(self, low, high, interval):
        self.wavelength = np.arange(low, high, interval)
        df = self.mat_df.reindex(self.wavelength)
        df = df.interpolate('spline', order=3)
        self.mat_df = df

    def better_bruggeman(self, n1, n2, percent_included):
        p = n1/n2
        b = .25*((3*percent_included-1)*(1/p-p)+p)
        z = b + (b**2 + .5)**0.5
        e = z*n1*n2
        return {"e": e, "n": e**0.5, 'conc': percent_included, "n1": n1, 'n2': n2}

    def brug_transform(self, df, layer, incl, percent):
        p = df[layer]/incl
        b = .25*((3*percent-1)*(1/p-p)+p)
        z = b + (b**2 + .5)**0.5
        e = z*df[layer]*incl
        n = e**.5
        df[layer] = n
    
    def run(self, wavelengths, void_percent):
        mat = self.mat_df.ix[wavelengths]
        mat = mat[self.layers]
        self.brug_transform(mat, self.layers[1], mat['Air'], void_percent)
        self.index_array = np.array(mat)
        theta0 = 45*sp.pi/180
        self.data = tmm.unpolarized_RT(self.index_array, self.thicknesses, theta0, wavelengths)
        
    def normalized(a, axis=-1, order=2):
        l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
        l2[l2==0] = 1
        return a / np.expand_dims(l2, axis)
    
    def get_R(self, wavelengths, thickness, void_percent):
        self.thicknesses[1] = thickness
        self.run(wavelengths, void_percent) 
        R = self.data['R']
        R -= min(R)
        R /= max(R)
        return R
        
    def RMSE(self, thickness, data):
        df = pd.DataFrame(data)
        self.get_R(thickness)
        model = pd.DataFrame(self.data['R'], index=self.wavelength)
        df = df.join(model, how='inner')
        n = len(df.index)
        return (sum((data-model)**2)/n)**0.5
    
    def norm(self, wavelength):
        df = self.df.ix[wavelength]
        df = df - df.min()
        df = df / df.max()
        return df
    
    def fit(self, wv, data):
        mod = lmfit.Model(self.get_R, ['wavelengths'], ['thickness','void_percent'])
        mod.set_param_hint('thickness', value = 120, min=50, max=250)
        mod.set_param_hint('void_percent', value = .15, min=0, max=1)
        
        weight = np.array(data.weight.ix[wv])
        R = data.series.ix[wv]
        result = mod.fit(R, wavelengths=wv, weights=weight)
        
        RMSE = (sp.sum(result.residual**2)/(result.residual.size-2))**0.5
        bf_values = result.best_values
        bf_str = 'thk: ' + str(round(bf_values['thickness'])) +", Void %: " + str(round(bf_values['void_percent']*100, 2))
        txt_spot = wv.min()-200 + (wv.max()-wv.min()) / 2

        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.text(txt_spot, .9, "RMSE: "+str(round(RMSE, 3)))
        ax.text(txt_spot, .85, bf_str)
        result.plot_fit(yerr=np.zeros(len(data.series.index)), data_kws ={'marker':'+'})

        plt.show()
        print(result.fit_report())

In [5]:
model = Model()
mat = model.mat_df[['Air', 'SiO2', 'TCO']]

In [26]:
def unpolarized_R(n_list, d_list, th_0, lam_vac):
    """
    Calculates reflected and transmitted power for unpolarized light.
    """

    s_data = tmm.coh_tmm('s',n_list, d_list, th_0, lam_vac)
    p_data = tmm.coh_tmm('p',n_list, d_list, th_0, lam_vac)
    R = (s_data['R'] + p_data['R']) / 2.
    return R

In [29]:
mat = model.mat_df[['Air', 'SiO2', 'TCO']]
d_list = [sp.inf, 120, sp.inf]
theta = 55*sp.pi/180
s = pd.Series(name='R')
for i in mat.index:
    index_array = np.array(mat.loc[i])
    r = unpolarized_R(index_array, d_list, theta, i)
    s = s.set_value(i, r)


In [30]:
from matplotlib import pyplot as plt
plt.plot(s.index, s)
plt.show()

In [10]:
mat = model.mat_df[['Air', 'SiO2', 'TCO']]
d_list = [sp.inf, 120, sp.inf]
theta = 55*sp.pi/180

In [118]:
%timeit get_RT(np.array(mat), d_list, theta, np.array(mat.index))


100 loops, best of 3: 6.97 ms per loop


In [16]:
%timeit tmm.coh_tmm('p', np.array(mat), d_list, theta, np.array(mat.index))

100 loops, best of 3: 3.33 ms per loop


In [116]:
def get_RT(n_array, d_list, theta, wavelengths):
    s_data = tmm.coh_tmm('s', n_array, d_list, theta, wavelengths)
    p_data = tmm.coh_tmm('p', n_array, d_list, theta, wavelengths)
    psi = np.arctan(abs(p_data['r']/s_data['r']))*180/sp.pi
    delta = -np.angle(-p_data['r']/s_data['r'])*180/sp.pi
    u_data = {'R':(s_data['R']+p_data['R'])/2, 'T':(s_data['R']+p_data['R'])/2}
    df = pd.DataFrame(np.array([s_data['R'], s_data['T'], p_data['R'], p_data['T'], u_data['R'], u_data['T'], psi, delta]).T,
        index=[wavelengths], columns=['Rs', 'Ts', 'Rp', 'Tp', 'R', 'T', 'Psi', 'Delta'])
    return df

def get_RT_tmm(n_array, d_list, theta, wavelengths):
    df = pd.DataFrame(columns=['Rs', 'Ts', 'Rp', 'Tp', 'R', 'T'])
    for i in range(len(wavelengths)):
        n_list = n_array[i]
        s_data = tmm.coh_tmm('s', n_list, d_list, theta, wavelengths[i])
        p_data = tmm.coh_tmm('p', n_list, d_list, theta, wavelengths[i])
        u_data = {'R':(s_data['R']+p_data['R'])/2, 'T':(s_data['R']+p_data['R'])/2}
        df.loc[wavelengths[i]] = [s_data['R'], s_data['T'], p_data['R'], p_data['T'], u_data['R'], u_data['T']]
        
    return df

In [122]:
x = tmm.coh_tmm('p', np.array(mat), d_list, theta, np.array(mat.index))

[        nan  0.00128139  0.00127327 ...,         nan         nan
         nan]


In [91]:
import tmm_core as tmm
dfme = get_RT(np.array(mat), d_list, theta, np.array(mat.index))

{'kz_list': array([[ 0.01201296+0.j        ,         nan       +nanj,
         0.02903770+0.00092217j],
       [ 0.01197305+0.j        ,  0.02605199+0.j        ,
         0.02892241+0.00090862j],
       [ 0.01193340+0.j        ,  0.02595878+0.j        ,
         0.02880963+0.00089247j],
       ..., 
       [        nan       +nanj,         nan       +nanj,
                nan       +nanj],
       [        nan       +nanj,         nan       +nanj,
                nan       +nanj],
       [        nan       +nanj,         nan       +nanj,
                nan       +nanj]]), 'n_list': array([[ 1.00000000+0.j        ,         nan+0.j        ,
         1.61020092+0.03791188j],
       [ 1.00000000+0.j        ,  1.49285186+0.j        ,
         1.60942815+0.03747315j],
       [ 1.00000000+0.j        ,  1.49257278+0.j        ,
         1.60872863+0.0369236j ],
       ..., 
       [        nan+0.j        ,         nan+0.j        ,         nan+0.j        ],
       [        nan+0.j        ,      

In [70]:
import tmm
%timeit get_RT_tmm(np.array(mat), d_list, theta, np.array(mat.index))

1 loops, best of 3: 2.77 s per loop


In [69]:
dftmm.applymap(lambda x: '%.5f' % x) == dfme.applymap(lambda x: '%.5f' % x)

,Rs,Ts,Rp,Tp,R,T
300,True,True,True,True,True,True
301,True,True,True,True,True,True
302,True,True,True,True,True,True
303,True,True,True,True,True,True
304,True,True,True,True,True,True
305,True,True,True,True,True,True
306,True,True,True,True,True,True
307,True,True,True,True,True,True
308,True,True,True,True,True,True
309,True,True,True,True,True,True


In [57]:
from matplotlib import pyplot as plt

In [66]:
d = 

In [ ]:
data = np.arange(1, 26)
index = np.arange(1, 6, .2).round(1)

x = pd.DataFrame(np.array([data, index.astype(int)]).T, columns=['data', 'index'])
x.mean(1)

In [ ]:
x = 3
y = -4

np.arctan(y/x)*180/np.pi

In [86]:
a = np.arange(6).reshape(2,3)
b = np.arange(6,12).reshape(2,3)
c = np.arange(12,18).reshape(2,3)
d = np.arange(18,24).reshape(2,3)

%timeit -o np.dstack((a,b,c,d)).reshape(a.shape + (2,2,))


100000 loops, best of 3: 8.16 µs per loop


<TimeitResult : 100000 loops, best of 3: 8.16 µs per loop>

In [87]:
def test(a, b, c, d):
    x = np.array([[a, b], [c, d]])
    y=np.rollaxis(np.rollaxis(x,-1),-1)
    return y

%timeit -o test(a, b, c, d)

The slowest run took 4.49 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 6.46 µs per loop


<TimeitResult : 100000 loops, best of 3: 6.46 µs per loop>

In [88]:
def test(a, b, c, d):
    ac =np.concatenate((a[:,:,np.newaxis],c[:,:,np.newaxis]),axis=-1)
    bd =np.concatenate((b[:,:,np.newaxis],d[:,:,np.newaxis]),axis=-1)
    abcd =np.concatenate((ac[:,:,:,np.newaxis],bd[:,:,:,np.newaxis]),axis=-1)
    return abcd

%timeit -o test(a, b, c, d)

100000 loops, best of 3: 6.24 µs per loop


<TimeitResult : 100000 loops, best of 3: 6.24 µs per loop>

In [77]:
def test(a, b, c, d):
    out = np.zeros(a.shape + (2,2))
    out[..., 0, 0] = a
    out[..., 0, 1] = b
    out[..., 1, 0] = c
    out[..., 1, 1] = d
    return out

%timeit test(a, b, c, d)

The slowest run took 7.78 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.41 µs per loop


In [78]:
%timeit test(a, b, c, d)

The slowest run took 6.73 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.41 µs per loop


In [54]:
def pretend(np_array):
    return sum(np_array)*np_array.name

df = pd.DataFrame(np.arange(16).reshape(8,2))
answer = df.apply(pretend,axis=1)
answer

0      0
1      5
2     18
3     39
4     68
5    105
6    150
7    203
dtype: int64